In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



Weaviate Key:root-user-key
OpenAI API Key: sk-dummy-key-for-local-testing
OpenAI URL: http://host.docker.internal:11434


# Setup


In [2]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_local(
    host="127.0.0.1",  # Localhost address
    port=8080,
    grpc_port=50051,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),  # Your local API key, if authentication is enabled
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Only needed if you use a model that requires this header
    }
)

print(client.is_ready())

True


## Create Tenant-ready collection

In [18]:
from weaviate.classes.config import Configure

if (client.collections.exists("Play")):
    client.collections.delete("Play")

client.collections.create(
    "Play",
    vector_config=Configure.Vectors.self_provided(),

    # multi_tenancy_config=Configure.multi_tenancy(True)

    multi_tenancy_config=Configure.multi_tenancy(
        enabled=True,
        auto_tenant_creation=True, #Assign to non-existant tenant will create
        auto_tenant_activation=True
    )
)

## Create tenants
> tenant name – must be made of alphanumeric characters (a-z, A-Z, 0-9), underscore (_), and hyphen (-), with a length between 1 and 64 characters'


In [19]:
from weaviate.classes.tenants import Tenant

play = client.collections.get("Play")

play.tenants.create([
    Tenant(name="ten_A"),
    Tenant(name="ten_B"),
    Tenant(name="ten_C"),
    Tenant(name="ten_D"),
])

## List Tenants

In [20]:
play.tenants.get()

{'ten_B': TenantOutput(name='ten_B', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_C': TenantOutput(name='ten_C', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_D': TenantOutput(name='ten_D', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_A': TenantOutput(name='ten_A', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

In [33]:
play.tenants.exists("ten_E")

True

## Access Tenants

In [22]:
# this will fail – multi-tenant collections require us to use tenants
play.aggregate.over_all()

WeaviateQueryError: Query call with protocol GRPC search failed with message <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "aggregate: class Play has multi-tenancy enabled, but request was without tenant"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"aggregate: class Play has multi-tenancy enabled, but request was without tenant", grpc_status:2, created_time:"2025-09-19T00:05:59.228736-05:00"}"
>.

In [23]:
tenA = play.with_tenant("ten_A")

tenA.aggregate.over_all()

AggregateReturn(properties={}, total_count=0)

### Insert data

In [ ]:
# tenA = client.collections.get("Play").with_tenant("ten_A")
play = client.collections.get("Play")
tenA = play.with_tenant("ten_A")

tenA.data.insert_many([
    {
       "title": "A book about vector databases"
    },
    {
       "title": "Tutorial for multimodal collections"
    },
])

tenA.aggregate.over_all()

AggregateReturn(properties={}, total_count=2)

### Query Example

In [25]:
from weaviate.classes.query import Filter

tenA = client.collections.get("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

{'title': 'A book about vector databases'}


### Delete Tenants

In [26]:
play.tenants.remove(["ten_D"])
play.tenants.get()

{'ten_A': TenantOutput(name='ten_A', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_B': TenantOutput(name='ten_B', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_C': TenantOutput(name='ten_C', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

### Update Tenants – Active & Inactive & Offloaded
Tenants can be:
* `Active` (default) - active tenants use  `HOT` resources (RAM)
* `Inacative` - inactive tenants cannot be searched on, their index is not loaded into memory, they don't use (RAM)
* `Offloaded` - offloaded tenants are moved to a cloud storage

> Tenant offloading, requires an extra configuration, which is out of scope for this workshop.<br/>
> You can learn more from [How-to: Configure - Tenant Offloading](https://weaviate.io/developers/weaviate/configuration/tenant-offloading)

### Deactivate – make tenant `Inactive`

In [27]:
from weaviate.classes.tenants import Tenant, TenantActivityStatus

play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.INACTIVE),
])

play.tenants.get()

{'ten_C': TenantOutput(name='ten_C', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_A': TenantOutput(name='ten_A', activityStatusInternal=<TenantActivityStatus.INACTIVE: 'INACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_B': TenantOutput(name='ten_B', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

**Cannot search `Inactive` tenants**

In [28]:
# tenA = client.collections.get("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

{'title': 'A book about vector databases'}


### Activate - make tenant `Active`

> You can't query an inactive tenant, but you can activate it.

In [29]:
play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.ACTIVE),
])

play.tenants.get()

{'ten_C': TenantOutput(name='ten_C', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_A': TenantOutput(name='ten_A', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'ten_B': TenantOutput(name='ten_B', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

In [30]:
response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

{'title': 'A book about vector databases'}


### Offload - make tenant `offloaded`

> Tenant offloading, requires an extra configuration, which is out of scope for this workshop.<br/>
> You can learn more from [How-to: Configure - Tenant Offloading](https://weaviate.io/developers/weaviate/configuration/tenant-offloading)

In [ ]:
# play.tenants.update([
#     Tenant(name="ten_A", activity_status=TenantActivityStatus.OFFLOADED),
# ])

# play.tenants.get()

UnexpectedStatusCodeError: Collection tenants may not have been updated properly for Play! Unexpected status code: 422, with response body: {'error': [{'message': "can't offload tenants, because offload-s3 module is not enabled"}]}.

## Clean up

In [17]:
client.collections.delete("Play")

## Don't close yet...

> You can try again with `auto_tenant_creation=True` and `auto_tenant_activation=True`

## Close the client

In [34]:
client.close()